In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os 
import json
import re
from collections import defaultdict
import processing_pdf
from gpt_summary import get_summaries

Define the input and output path.

In [6]:
# path to the folder containing PDF files (1512, removed researchpaper1.pdf)
dataset_path = "F:\Datasets/"
# output folder and file name
output_path = "dataset"
output_file_name = "dataset.json"
processing_pdf.clear_processed_folder(output_path)

Step 0: Remove duplicated files, only keep the papers (941) with newest version

In [7]:
# Function to extract the base filename without the version suffix
def get_base_filename(filename):
    match = re.match(r'^(.*?)v\d\.pdf+$', filename)
    if match:
        return match.group(1)
    else:
        return filename

# Function to find the newest version of each file
def find_newest_versions(folder):
    files_by_base_name = defaultdict(list)

    # Group files by base filename
    for filename in os.listdir(folder):
        base_name = get_base_filename(filename)
        files_by_base_name[base_name].append(filename)

    # Find the newest version of each file
    newest_versions = []
    for base_name, filenames in files_by_base_name.items():
        newest_version = max(filenames, key=lambda x: int(re.search(r'v(\d+)\.pdf$', x).group(1)))
        newest_versions.append(newest_version)

    return newest_versions

# Function to remove duplicated files
def remove_duplicates(folder):
    newest_versions = find_newest_versions(folder)

    for filename in os.listdir(folder):
        if filename not in newest_versions:
            file_path = os.path.join(folder, filename)
            if os.path.isfile(file_path):
                os.remove(file_path)

remove_duplicates(dataset_path)

Step 1: Breakdown PDF content by sections and subsections.

In [8]:
def process_one_pdf(file_path):
    doc, total_text, _ = processing_pdf.open_file(file_path)
    table_of_content = doc.get_toc()

    if len(table_of_content) > 0:
        print("Auto generated table of content:")
        display(table_of_content)
        # separate content into sections
        _, json_dict = processing_pdf.separate_content(total_text, table_of_content)
        return json_dict
    # some papers have not table of content
    #if len(table_of_content) == 0:
    #    print("The paper has not table of content. Need to use regular expression to map table of content.")
    #    table_of_content = processing_pdf.auto_find_toc(doc)
    #    display(table_of_content)
    
    return {}

In [9]:
# number of PDFs used for training data preparation
dataset_limit = 100

file_count = 0
all_json_dicts = []
# Loop through files in the folder
for file_name in os.listdir(dataset_path):
    file_path = os.path.join(dataset_path, file_name)
    # Check if it's a file and if it has a ".pdf" extension
    if os.path.isfile(file_path) and file_name.endswith('.pdf'):
        # Process the PDF file
        print("Processing PDF file:", file_name)
        json_dict = process_one_pdf(file_path)
        if len(json_dict) > 0:
            all_json_dicts.extend(list(json_dict.values()))
            file_count += 1
            print("Done with PDF file:", file_name)
            print("# of sections:", len(json_dict.values()))
            print("Total # of sections:", len(all_json_dicts), "Total # of files:", file_count)
            print(60*"=")
        
        # Terminate when reaching dataset limit
        if file_count >= dataset_limit:
            break

# get_summaries(all_json_dicts)
json_list = json.dumps(all_json_dicts)
full_name = "dataset/dataset.json"
with open(full_name, "w") as jsonfile: 
    jsonfile.write(json_list)

Processing PDF file: 1901.00001v1.pdf
The paper has not table of content. Need to use regular expression to map table of content.
starting looking for all the sections...


[[1, '1 \nIntroduction ', 1],
 [1, '1 \nFine \nFine ', 4],
 [1, '2 \nFine \nCoarse ', 4],
 [1, '3 \nCoarse \nFine ', 4],
 [1, '4 \nCoarse \nCoarse ', 4],
 [1, '4 \nResults ', 5],
 [1, '5 \nDiscussion ', 8],
 [1, '6 \nConclusions ', 9]]

starting looking for all the sections according to the provided section title info...
Done with PDF file: 1901.00001v1.pdf
# of sections: 9
Total # of sections: 9 Total # of files: 1
Processing PDF file: 1901.00002v1.pdf
Auto generated table of content:


[[1, 'I Introduction', 1],
 [1, 'II  Modeling', 2],
 [2, 'A Assume f(t) proportional to (e+pe)', 3],
 [2, 'B Assume f(t) proportional to e', 5],
 [1, 'III Comparison with the early universe', 5],
 [1, 'IV Summary', 6],
 [1, ' References', 6]]

starting looking for all the sections according to the provided section title info...
Done with PDF file: 1901.00002v1.pdf
# of sections: 6
Total # of sections: 15 Total # of files: 2
Processing PDF file: 1901.00003v3.pdf
The paper has not table of content. Need to use regular expression to map table of content.
starting looking for all the sections...


...

starting looking for all the sections according to the provided section title info...
Done with PDF file: 1901.00097v1.pdf
# of sections: 6
Total # of sections: 636 Total # of files: 98
Processing PDF file: 1901.00098v1.pdf
The paper has not table of content. Need to use regular expression to map table of content.
starting looking for all the sections...


[[1, '1. Introduction', 1],
 [1, '2. Related Work\nPrivacy-Preserving\nVisual', 2],
 [2, '3.1. Formulation', 3],
 [1, '4. Training', 4],
 [1, '5. Experiments', 5],
 [2, '5.1. Datasets\nWe have evaluated our obfuscation method with three', 5],
 [2, '5.3. Baselines', 5],
 [2, '5.4. Image Classiﬁcation', 5],
 [1, '1.\nDataset', 6],
 [2, '5.5. Attribute Classiﬁcation', 7],
 [2, '5.6. Facial Landmark Detection', 7],
 [1, '6. Conclusion', 8],
 [1, '32\nCBR', 9],
 [1, '32\nCBR', 9],
 [1, '32\nCS', 9],
 [1, '32\nCBR', 9],
 [1, '32\nCBR', 9],
 [1, '32\nCBR', 9],
 [1, '32\nCT', 9],
 [1, '32\nCBR', 9],
 [1, '32\nCBR', 9],
 [1, '32\nCBR', 9],
 [1, '32\nCBR', 9],
 [1, '32\nCS', 9],
 [1, '16\nCBL', 9],
 [1, '32\nCBL', 9],
 [1, '64\nCBL', 9],
 [1, '128\nCBR', 9],
 [1, '128\nCBDR', 9],
 [1, '256\nCBDR', 9],
 [1, '128\nCBDR', 9],
 [1, '64\nCBR', 9],
 [1, '32\nCS', 9],
 [1, '5.\nLayer\nKernel\nStride\nPadding\nDilation\nOut\nCBR', 9],
 [1, '8\nCBR', 9],
 [1, '16\nCBR', 9],
 [1, '32\nCBR', 9],
 [1, '32\n

starting looking for all the sections according to the provided section title info...
Done with PDF file: 1901.00098v1.pdf
# of sections: 23
Total # of sections: 659 Total # of files: 99
Processing PDF file: 1901.00099v2.pdf
The paper has not table of content. Need to use regular expression to map table of content.
starting looking for all the sections...


[[1, '1959 NE Pacific St ', 1],
 [1, '2 \nAbstract \n ', 2],
 [1, '1. Introduction \n ', 3],
 [1, '8. Conclusions \n ', 18],
 [1, 'I. Parkinson Study ', 22],
 [2, 'D. Posthuma ', 26]]

starting looking for all the sections according to the provided section title info...
Done with PDF file: 1901.00099v2.pdf
# of sections: 6
Total # of sections: 665 Total # of files: 100


Step 2: Generate GPT summary for each section and subsection as ground truth.